In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "vnHqPK9iFNbW"
      },
      "outputs": [],
      "source": [
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "from io import BytesIO\n",
        "from PIL import Image\n",
        "\n",
        "# Physical constants\n",
        "hbar = 1.0545718e-34\n",
        "m_e = 9.1093837e-31\n",
        "\n",
        "# PIB Model Functions\n",
        "def pib_wavefunction(x, n, L):\n",
        "    return np.sqrt(2 / L) * np.sin(n * np.pi * x / L)\n",
        "\n",
        "def pib_energy(n, L):\n",
        "    return (hbar**2 * np.pi**2 * n**2) / (2 * m_e * L**2)\n",
        "\n",
        "def plot_pib(L, n_max=3, num_particles=1):\n",
        "    x = np.linspace(0, L, 1000)\n",
        "    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))\n",
        "    max_energy = pib_energy(n_max, L)\n",
        "    for n in range(1, n_max + 1):\n",
        "        psi = pib_wavefunction(x, n, L)\n",
        "        energy = pib_energy(n, L)\n",
        "        ax1.plot(x * 1e9, psi + energy / max_energy, label=f\"n={n}\")\n",
        "    ax1.set_title(\"PIB Wavefunctions\")\n",
        "    ax1.set_xlabel(\"x (nm)\")\n",
        "    ax1.set_ylabel(\"ψ(x) + E (scaled)\")\n",
        "    ax1.legend()\n",
        "    energies = []\n",
        "    particles_filled = 0\n",
        "    for n in range(1, n_max + 1):\n",
        "        energy = pib_energy(n, L)\n",
        "        energies.append(energy)\n",
        "        ax2.axhline(energy, linestyle='-', label=f\"n={n}\")\n",
        "        if particles_filled < num_particles:\n",
        "            remaining = min(2, num_particles - particles_filled)\n",
        "            ax2.text(0.5, energy, f\"{remaining} particle(s)\", verticalalignment='bottom')\n",
        "            particles_filled += remaining\n",
        "    ax2.set_title(\"PIB Energy Levels\")\n",
        "    ax2.set_ylabel(\"Energy (J)\")\n",
        "    ax2.legend()\n",
        "    if energies:\n",
        "        ax2.set_ylim(0, max(energies) * 1.1)\n",
        "    else:\n",
        "        ax2.set_ylim(0, 1e-18)\n",
        "    plt.tight_layout()\n",
        "    plt.show()\n",
        "    return fig\n",
        "\n",
        "# POR Model Functions\n",
        "def por_wavefunction(phi, ml):\n",
        "    return (1 / np.sqrt(2 * np.pi)) * np.exp(1j * ml * phi)\n",
        "\n",
        "def por_energy(ml, R):\n",
        "    return (ml**2 * hbar**2) / (2 * m_e * R**2)\n",
        "\n",
        "def plot_por(R, n_max=3, num_particles=1):\n",
        "    phi = np.linspace(0, 2 * np.pi, 1000)\n",
        "    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))\n",
        "    max_energy = por_energy(n_max, R)\n",
        "    for ml in range(0, n_max + 1):\n",
        "        psi_real = np.real(por_wavefunction(phi, ml))\n",
        "        energy = por_energy(ml, R)\n",
        "        ax1.plot(phi, psi_real + energy / max_energy, label=f\"ml={ml}\")\n",
        "    ax1.set_title(\"POR Wavefunctions (Real Part)\")\n",
        "    ax1.set_xlabel(\"φ (rad)\")\n",
        "    ax1.set_ylabel(\"Re[ψ(φ)] + E (scaled)\")\n",
        "    ax1.legend()\n",
        "    energies = []\n",
        "    particles_filled = 0\n",
        "    for ml in range(0, n_max + 1):\n",
        "        energy = por_energy(ml, R)\n",
        "        energies.append(energy)\n",
        "        ax2.axhline(energy, linestyle='-', label=f\"ml=±{ml}\" if ml > 0 else \"ml=0\")\n",
        "        if particles_filled < num_particles:\n",
        "            remaining = min(2 if ml == 0 else 4, num_particles - particles_filled)\n",
        "            ax2.text(0.5, energy, f\"{remaining} particle(s)\", verticalalignment='bottom')\n",
        "            particles_filled += remaining\n",
        "    ax2.set_title(\"POR Energy Levels\")\n",
        "    ax2.set_ylabel(\"Energy (J)\")\n",
        "    ax2.legend()\n",
        "    if energies:\n",
        "        ax2.set_ylim(-max(energies) * 0.1, max(energies) * 1.1)\n",
        "    else:\n",
        "        ax2.set_ylim(-1e-20, 1e-20)\n",
        "    plt.tight_layout()\n",
        "    plt.show()\n",
        "    return fig\n",
        "\n",
        "# Energy Calculation\n",
        "def calculate_energies(L, R, num_particles=6):\n",
        "    pib_total_energy = 0\n",
        "    particles_filled = 0\n",
        "    n = 1\n",
        "    while particles_filled < num_particles:\n",
        "        energy = pib_energy(n, L)\n",
        "        particles_to_add = min(2, num_particles - particles_filled)\n",
        "        pib_total_energy += particles_to_add * energy\n",
        "        particles_filled += particles_to_add\n",
        "        n += 1\n",
        "    por_total_energy = 0\n",
        "    particles_filled = 0\n",
        "    ml = 0\n",
        "    while particles_filled < num_particles:\n",
        "        energy = por_energy(ml, R)\n",
        "        particles_to_add = min(2 if ml == 0 else 4, num_particles - particles_filled)\n",
        "        por_total_energy += particles_to_add * energy\n",
        "        particles_filled += particles_to_add\n",
        "        ml += 1\n",
        "    ase = pib_total_energy - por_total_energy\n",
        "    return pib_total_energy, por_total_energy, ase"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import ipywidgets as widgets\n",
        "from IPython.display import display\n",
        "\n",
        "L_input = widgets.FloatText(value=1.0, description='L (nm):')\n",
        "R_input = widgets.FloatText(value=0.139, description='R (nm):')\n",
        "n_max_input = widgets.IntText(value=3, description='n_max:')\n",
        "particles_input = widgets.IntText(value=6, description='Particles:')\n",
        "plot_button = widgets.Button(description='Plot')\n",
        "\n",
        "display(L_input, R_input, n_max_input, particles_input, plot_button)\n",
        "\n",
        "def on_plot_button_clicked(b):\n",
        "    L = L_input.value * 1e-9\n",
        "    R = R_input.value * 1e-9\n",
        "    n_max = n_max_input.value\n",
        "    num_particles = particles_input.value\n",
        "    if L <= 0 or R <= 0 or n_max < 0 or num_particles < 1:\n",
        "        print(\"Error: Invalid input\")\n",
        "        return\n",
        "    plot_pib(L, n_max, num_particles)\n",
        "    plot_por(R, n_max, num_particles)\n",
        "    pib_energy, por_energy, ase = calculate_energies(L, R, num_particles)\n",
        "    print(f\"PIB Energy ({num_particles} particles): {pib_energy:.2e} J\")\n",
        "    print(f\"POR Energy ({num_particles} particles): {por_energy:.2e} J\")\n",
        "    print(f\"Aromatic Stabilization Energy: {ase:.2e} J\")\n",
        "\n",
        "plot_button.on_click(on_plot_button_clicked)"
      ]
    }
  ]
}

In [ ]:
import ipywidgets as widgets
from IPython.display import display

L_input = widgets.FloatText(value=1.0, description='L (nm):')
R_input = widgets.FloatText(value=0.139, description='R (nm):')
n_max_input = widgets.IntText(value=3, description='n_max:')
particles_input = widgets.IntText(value=6, description='Particles:')
plot_button = widgets.Button(description='Plot')

display(L_input, R_input, n_max_input, particles_input, plot_button)

def on_plot_button_clicked(b):
    L = L_input.value * 1e-9
    R = R_input.value * 1e-9
    n_max = n_max_input.value
    num_particles = particles_input.value
    if L <= 0 or R <= 0 or n_max < 0 or num_particles < 1:
        print("Error: Invalid input")
        return
    plot_pib(L, n_max, num_particles)
    plot_por(R, n_max, num_particles)
    pib_energy, por_energy, ase = calculate_energies(L, R, num_particles)
    print(f"PIB Energy ({num_particles} particles): {pib_energy:.2e} J")
    print(f"POR Energy ({num_particles} particles): {por_energy:.2e} J")
    print(f"Aromatic Stabilization Energy: {ase:.2e} J")

plot_button.on_click(on_plot_button_clicked)